<a href="https://colab.research.google.com/github/Adityaaz10/Pnuemonia_detection_model/blob/main/Pneumonia_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu121
!pip -q install transformers datasets accelerate timm
!pip -q install kaggle pydicom pillow opencv-python scikit-learn matplotlib seaborn grad-cam gradio


In [ ]:
import torch; torch.cuda.is_available()


True

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"adimukherjee10","key":"3651aa66b080724db637883a25ed89c8"}'}

In [ ]:
import os, shutil
os.makedirs('/root/.kaggle', exist_ok=True)
shutil.move('/content/kaggle.json', '/root/.kaggle/kaggle.json')
os.chmod('/root/.kaggle/kaggle.json', 0o600)


In [ ]:
!kaggle competitions download -c rsna-pneumonia-detection-challenge -p /content/rsna
import zipfile, glob
for z in glob.glob('/content/rsna/*.zip'):
    with zipfile.ZipFile(z, 'r') as zip_ref:
        zip_ref.extractall('/content/rsna')


rsna-pneumonia-detection-challenge.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!pip -q install pydicom

import pandas as pd, numpy as np, os, cv2, pydicom
from tqdm import tqdm

DICOM_DIR = '/content/rsna/stage_2_train_images'
LABELS_CSV = '/content/rsna/stage_2_train_labels.csv'
OUT_DIR = '/content/rsna_png'
os.makedirs(f'{OUT_DIR}/images', exist_ok=True)

labels = pd.read_csv(LABELS_CSV)
img_labels = labels.groupby('patientId')['Target'].max().reset_index()
img_labels['label'] = img_labels['Target'].astype(int)
img_labels = img_labels[['patientId','label']]

# LIMIT to a manageable subset first (adjust up later)
MAX_IMAGES = 12000  # try 4000 if still OOM; 12000 if stable
img_labels = img_labels.sample(n=min(MAX_IMAGES, len(img_labels)), random_state=42)

# Low resolution to save memory; try (160,160) or (128,128) if still OOM
RES = 160

def dicom_to_png(path, size=(RES, RES)):
    d = pydicom.dcmread(path)
    arr = d.pixel_array.astype(np.float32)
    # normalize 0-255
    arr -= arr.min()
    if arr.max() > 0:
        arr /= arr.max()
    arr = (arr * 255.0).clip(0,255).astype(np.uint8)
    arr = cv2.resize(arr, size, interpolation=cv2.INTER_AREA)
    img3 = np.stack([arr, arr, arr], axis=-1)
    return img3

paths, ys = [], []
for pid, y in tqdm(img_labels.values):
    p = os.path.join(DICOM_DIR, f'{pid}.dcm')
    if not os.path.exists(p):
        continue
    img = dicom_to_png(p)
    outp = os.path.join(OUT_DIR, 'images', f'{pid}.png')
    cv2.imwrite(outp, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    paths.append(outp)
    ys.append(int(y))

meta = pd.DataFrame({'image_path': paths, 'label': ys})
meta.to_csv(f'{OUT_DIR}/train_meta.csv', index=False)
meta['label'].value_counts(), len(meta)


100%|██████████| 12000/12000 [02:46<00:00, 72.12it/s]


(label
 0    9248
 1    2752
 Name: count, dtype: int64,
 12000)

In [ ]:
from datasets import Dataset
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd

meta = pd.read_csv('/content/rsna_png/train_meta.csv')
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=42)
train_idx, val_idx = next(sss.split(meta['image_path'], meta['label']))
train_df = meta.iloc[train_idx].reset_index(drop=True)
val_df = meta.iloc[val_idx].reset_index(drop=True)

ds_train = Dataset.from_pandas(train_df)
ds_val = Dataset.from_pandas(val_df)
len(ds_train), len(ds_val)


(10200, 1800)

In [ ]:
import timm, torch
import torch.nn as nn
from PIL import Image
from torchvision import transforms

# Image transforms (match RES)
RES = 224  # set same as preprocessing
train_tfms = transforms.Compose([
    transforms.Resize((RES, RES)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.25,0.25,0.25])
])
val_tfms = transforms.Compose([
    transforms.Resize((RES, RES)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.25,0.25,0.25])
])

# Lazy loader functions
def load_image(example):
    example['image'] = example['image_path']
    return example

ds_train = ds_train.map(load_image)
ds_val = ds_val.map(load_image)

# Minimal collate to load+transform per batch
def collate_fn(batch, train=True):
    imgs = []
    labels = []
    for ex in batch:
        img = Image.open(ex['image']).convert('RGB')
        img = (train_tfms if train else val_tfms)(img)
        imgs.append(img)
        labels.append(int(ex['label']))
    return {'pixel_values': torch.stack(imgs), 'labels': torch.tensor(labels, dtype=torch.long)}

# Model
num_classes = 2
model = timm.create_model('resnet18', pretrained=True, num_classes=num_classes)
model = model.cuda()


Map:   0%|          | 0/10200 [00:00<?, ? examples/s]

Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

In [ ]:
import os, zipfile

KAGGLE_DATASET = 'himanshu007121/coughclassifier-trial'
DATA_DIR = './coughclassifier_trial'
os.makedirs(DATA_DIR, exist_ok=True)

os.system(f'kaggle datasets download -d {KAGGLE_DATASET} -p {DATA_DIR}')
for file in os.listdir(DATA_DIR):
    if file.endswith('.zip'):
        with zipfile.ZipFile(os.path.join(DATA_DIR, file), 'r') as zip_ref:
            zip_ref.extractall(DATA_DIR)


In [ ]:
print(meta.columns)



Index(['chroma_stft', 'rmse', 'spectral_centroid', 'spectral_bandwidth',
       'rolloff', 'zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4',
       'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11',
       'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18',
       'mfcc19', 'mfcc20', 'label'],
      dtype='object')


In [ ]:
LABEL_CSV = os.path.join(DATA_DIR, 'Smote_data.csv')
meta = pd.read_csv(LABEL_CSV)
print(meta.columns)
print(meta.head())

# Suppose columns are 'audio', 'label' where label: 1=positive/covid, 0=negative/healthy
filtered = meta.copy()
filtered.to_csv(os.path.join(DATA_DIR, 'filtered_labels.csv'), index=False)


Index(['chroma_stft', 'rmse', 'spectral_centroid', 'spectral_bandwidth',
       'rolloff', 'zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4',
       'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11',
       'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18',
       'mfcc19', 'mfcc20', 'label'],
      dtype='object')
   chroma_stft      rmse  spectral_centroid  spectral_bandwidth   rolloff  \
0     0.975000 -0.225520          -1.133697           -1.419965 -1.341559   
1     1.138436 -1.085691           0.458562            0.432626  0.072958   
2     0.729820 -0.463494           1.126956            0.996399  1.257855   
3     0.079311 -0.968320          -0.561011            0.281959 -0.012494   
4    -0.070381 -0.531515          -0.842920           -0.657277 -0.780357   

   zero_crossing_rate     mfcc1     mfcc2     mfcc3     mfcc4  ...    mfcc12  \
0           -0.632042 -0.583560  0.898607 -0.365554 -0.259207  ... -0.657483   
1           -0.05771

In [ ]:
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader

SAMPLE_RATE = 22050
N_MELS = 128
SEG_LEN = 128

class CoughAudioDataset(Dataset):
    def __init__(self, audio_dir, label_csv):
        self.meta = pd.read_csv(label_csv)
        self.audio_dir = audio_dir
        self.sr = SAMPLE_RATE
        self.n_mels = N_MELS
        self.seg_len = SEG_LEN
        self.mel_transform = torchaudio.transforms.MelSpectrogram(sample_rate=self.sr, n_mels=self.n_mels)
        self.db_transform = torchaudio.transforms.AmplitudeToDB()
    def __len__(self):
        return len(self.meta)
    def pad_or_crop(self, spec):
        if spec.shape[-1] < self.seg_len:
            pad_amt = self.seg_len - spec.shape[-1]
            spec = torch.nn.functional.pad(spec, (0, pad_amt))
        else:
            spec = spec[:, :self.seg_len]
        return spec
    def __getitem__(self, idx):
        fname, label = self.meta.iloc[idx], int(self.meta.iloc[idx]['label'])
        path = os.path.join(self.audio_dir, fname)
        wav, sr = torchaudio.load(path)
        if sr != self.sr:
            wav = torchaudio.functional.resample(wav, orig_freq=sr, new_freq=self.sr)
        if wav.shape[0] > 1:
            wav = torch.mean(wav, dim=0, keepdim=True)
        mel = self.mel_transform(wav)
        mel_db = self.db_transform(mel)
        mel_db = mel_db.squeeze()
        mel_db = self.pad_or_crop(mel_db)
        mel_db = mel_db.unsqueeze(0)
        return mel_db, torch.tensor(label, dtype=torch.long)

dataset = CoughAudioDataset(DATA_DIR, os.path.join(DATA_DIR, 'filtered_labels.csv'))
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

/usr/local/lib/python3.12/dist-packages/torchaudio/functional/functional.py:585: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [ ]:
from torchvision.models import resnet18
import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = resnet18(pretrained=True)
model.conv1 = nn.Conv2d(1, model.conv1.out_channels, kernel_size=model.conv1.kernel_size,
                        stride=model.conv1.stride, padding=model.conv1.padding, bias=False)
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

import torch.optim as optim
from tqdm import tqdm

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
for epoch in range(5):
    model.train()
    total_loss = 0.0
    for X, y in tqdm(dataloader):
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(X)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * X.size(0)
    print(f"Epoch {epoch+1}: avg loss = {total_loss / len(dataset):.4f}")


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0%|          | 0/37 [00:00<?, ?it/s]


TypeError: join() argument must be str, bytes, or os.PathLike object, not 'Series'

In [ ]:
import math, numpy as np
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score


BATCH = 16  # try 16; if OOM, set 8 or 4
ACCUM = 2   # gradient accumulation steps (effective batch = BATCH*ACCUM)
EPOCHS = 3
LR = 3e-4


train_loader = DataLoader(ds_train, batch_size=BATCH, shuffle=True, collate_fn=lambda x: collate_fn(x, True), num_workers=2, pin_memory=True)
val_loader   = DataLoader(ds_val,   batch_size=BATCH, shuffle=False, collate_fn=lambda x: collate_fn(x, False), num_workers=2, pin_memory=True)


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-2)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS*len(train_loader))


best_auc = -1
for epoch in range(EPOCHS):
    model.train()
    optimizer.zero_grad(set_to_none=True)
    running_loss = 0.0
    for step, batch in enumerate(train_loader):
        x = batch['pixel_values'].cuda(non_blocking=True)
        y = batch['labels'].cuda(non_blocking=True)
        logits = model(x)
        loss = criterion(logits, y) / ACCUM
        loss.backward()
        if (step+1) % ACCUM == 0:
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()
        running_loss += loss.item() * ACCUM
        if (step+1) % 100 == 0:
            print(f"epoch {epoch+1} step {step+1}/{len(train_loader)} loss {running_loss/(step+1):.4f}")


    # Eval
    model.eval()
    all_probs, all_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            x = batch['pixel_values'].cuda(non_blocking=True)
            y = batch['labels'].cuda(non_blocking=True)
            logits = model(x)
            probs = torch.softmax(logits, dim=1)[:,1].detach().cpu().numpy()
            all_probs.append(probs)
            all_labels.append(y.detach().cpu().numpy())
    all_probs = np.concatenate(all_probs)
    all_labels = np.concatenate(all_labels)
    preds = (all_probs >= 0.5).astype(int)
    acc = accuracy_score(all_labels, preds)
    prec, rec, f1, _ = precision_recall_fscore_support(all_labels, preds, average='binary')
    try:
        auc = roc_auc_score(all_labels, all_probs)
    except:
        auc = float('nan')
    print(f"E{epoch+1}: acc={acc:.3f} rec={rec:.3f} f1={f1:.3f} auc={auc:.3f}")
    if auc > best_auc:
        best_auc = auc
        torch.save(model.state_dict(), '/content/best_resnet18.pth')
print('Best AUC:', best_auc)

epoch 1 step 100/638 loss 0.4795
epoch 1 step 200/638 loss 0.4601
epoch 1 step 300/638 loss 0.4463
epoch 1 step 400/638 loss 0.4373
epoch 1 step 500/638 loss 0.4285
epoch 1 step 600/638 loss 0.4273
E1: acc=0.828 rec=0.545 f1=0.592 auc=0.857
epoch 2 step 100/638 loss 0.3857
epoch 2 step 200/638 loss 0.3841
epoch 2 step 300/638 loss 0.3891
epoch 2 step 400/638 loss 0.3902
epoch 2 step 500/638 loss 0.3872
epoch 2 step 600/638 loss 0.3866
E2: acc=0.828 rec=0.593 f1=0.613 auc=0.861
epoch 3 step 100/638 loss 0.3728
epoch 3 step 200/638 loss 0.3729
epoch 3 step 300/638 loss 0.3700
epoch 3 step 400/638 loss 0.3675
epoch 3 step 500/638 loss 0.3653
epoch 3 step 600/638 loss 0.3597
E3: acc=0.827 rec=0.598 f1=0.613 auc=0.859
Best AUC: 0.8612871859239462


In [ ]:
!pip -q install --upgrade grad-cam


In [ ]:

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
import numpy as np, cv2
from PIL import Image
import torch

model.eval()

# Choose a proper conv layer for ResNet18 (timm)
target_layers = [model.layer4[-1].conv2]  # conv layer, not the whole block

# Prepare one image
sample_path = val_df.sample(1, random_state=0).iloc[0]['image_path']
img = Image.open(sample_path).convert('RGB').resize((RES, RES))
img_np = np.array(img).astype(np.float32) / 255.0

# Transform (same as your val_tfms but keep a single tensor)
x = val_tfms(img).unsqueeze(0)
x = x.cuda()  # move to GPU

# Initialize GradCAM (no use_cuda argument in new versions)
cam = GradCAM(model=model, target_layers=target_layers)

# Run CAM for the positive class (index 1 = Pneumonia)
grayscale_cam = cam(input_tensor=x, targets=[ClassifierOutputTarget(1)])

# Convert and overlay
grayscale = grayscale_cam[0]  # first (and only) image in batch
vis = show_cam_on_image(img_np, grayscale, use_rgb=True)
cv2.imwrite('/content/cam_resnet18.png', cv2.cvtColor(vis, cv2.COLOR_RGB2BGR))
'/content/cam_resnet18.png'


'/content/cam_resnet18.png'

In [ ]:
import gradio as gr

labels = ['Normal', 'Pneumonia']
model.eval()

def predict(img):
    img = Image.fromarray(img).convert('RGB')
    x = val_tfms(img).unsqueeze(0).cuda()
    with torch.no_grad():
        logits = model(x)
        probs = torch.softmax(logits, dim=1).squeeze(0).cpu().numpy()
    return {labels[i]: float(probs[i]) for i in range(2)}

demo = gr.Interface(fn=predict, inputs=gr.Image(type='numpy'), outputs=gr.Label(num_top_classes=2), title='Pneumonia Detector (ResNet18)')
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://02dad1af5153c58e45.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import os; os.path.exists('/content/best_resnet18.pth')


True

In [ ]:
import timm, torch, torch.nn as nn
from PIL import Image
from torchvision import transforms
import numpy as np

num_classes = 2
model = timm.create_model('resnet18', pretrained=False, num_classes=num_classes).cuda()
model.load_state_dict(torch.load('/content/best_resnet18.pth', map_location='cuda'))
model.eval()

RES = 224
val_tfms = transforms.Compose([
    transforms.Resize((RES, RES)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.25,0.25,0.25]),
])

sample_path = val_df.sample(1, random_state=0).iloc[0]['image_path']
img = Image.open(sample_path).convert('RGB')
x = val_tfms(img).unsqueeze(0).cuda()
with torch.no_grad():
    probs = torch.softmax(model(x), dim=1).squeeze(0).cpu().numpy()
print({'Normal': float(probs[0]), 'Pneumonia': float(probs[1])})


{'Normal': 0.8946426510810852, 'Pneumonia': 0.105357326567173}
